In [1]:
import numpy as np
from keras.models import Model
from keras.applications import Xception
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

from keras.applications import imagenet_utils
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
n_classes = len(np.unique(y_train))
y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)

X_train = X_train.astype('float32')/255.
X_test = X_test.astype('float32')/255.

In [4]:
xception_model = Xception(weights='imagenet', include_top=False)

83689472/83683744 [==============================] - 49s 1us/step


In [5]:
x = xception_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
out = Dense(10, activation='softmax')(x)

In [6]:
model = Model(inputs=xception_model.input, outputs=out)

In [7]:
for layer in xception_model.layers:
    layer.trainable = False

In [8]:
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [9]:
callbacks = [EarlyStopping(monitor='val_acc', patience=5, verbose=0)]


In [10]:
n_epochs = 10
batch_size = 50
history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_split=0.2, verbose=1, callbacks=callbacks)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 525s 13ms/step - loss: 2.2701 - acc: 0.1518 - val_loss: 2.2575 - val_acc: 0.1704
Epoch 2/10
40000/40000 [==============================] - 489s 12ms/step - loss: 2.2182 - acc: 0.1754 - val_loss: 2.2128 - val_acc: 0.1878
Epoch 3/10
40000/40000 [==============================] - 487s 12ms/step - loss: 2.1960 - acc: 0.1808 - val_loss: 2.2112 - val_acc: 0.1714
Epoch 4/10
40000/40000 [==============================] - 489s 12ms/step - loss: 2.1766 - acc: 0.1914 - val_loss: 2.2092 - val_acc: 0.1813
Epoch 5/10
40000/40000 [==============================] - 486s 12ms/step - loss: 2.1692 - acc: 0.1927 - val_loss: 2.1965 - val_acc: 0.1904
Epoch 6/10
40000/40000 [==============================] - 493s 12ms/step - loss: 2.1608 - acc: 0.1962 - val_loss: 2.1997 - val_acc: 0.1899
Epoch 7/10
40000/40000 [==============================] - 485s 12ms/step - loss: 2.1568 - acc: 0.1988 - val_loss: 2.

In [11]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv1_bn
3 block1_conv1_act
4 block1_conv2
5 block1_conv2_bn
6 block1_conv2_act
7 block2_sepconv1
8 block2_sepconv1_bn
9 block2_sepconv2_act
10 block2_sepconv2
11 block2_sepconv2_bn
12 conv2d_1
13 block2_pool
14 batch_normalization_1
15 add_1
16 block3_sepconv1_act
17 block3_sepconv1
18 block3_sepconv1_bn
19 block3_sepconv2_act
20 block3_sepconv2
21 block3_sepconv2_bn
22 conv2d_2
23 block3_pool
24 batch_normalization_2
25 add_2
26 block4_sepconv1_act
27 block4_sepconv1
28 block4_sepconv1_bn
29 block4_sepconv2_act
30 block4_sepconv2
31 block4_sepconv2_bn
32 conv2d_3
33 block4_pool
34 batch_normalization_3
35 add_3
36 block5_sepconv1_act
37 block5_sepconv1
38 block5_sepconv1_bn
39 block5_sepconv2_act
40 block5_sepconv2
41 block5_sepconv2_bn
42 block5_sepconv3_act
43 block5_sepconv3
44 block5_sepconv3_bn
45 add_4
46 block6_sepconv1_act
47 block6_sepconv1
48 block6_sepconv1_bn
49 block6_sepconv2_act
50 block6_sepconv2
51 block6_sepconv2_bn
52 block6_sepcon

In [12]:
for layer in model.layers[:115]:
    layer.trainable = False
for layer in model.layers[115:]:
    layer.trainable = True

In [13]:
opt_finetune = Adam()
model.compile(optimizer=opt_finetune, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [14]:
history_finetune = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_split=0.2, verbose=1, callbacks=callbacks)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
 1900/40000 [>.............................] - ETA: 28:02 - loss: 2.2949 - acc: 0.1421

KeyboardInterrupt: 